# jakluz-de3.1.5 Data reseach
In this notebook I intend to research the input data from [Kaggle](https://www.kaggle.com/datasets/vikasukani/loan-eligible-dataset) to figure out, how can I transfor the data into the star schema.

In [55]:
import pandas as pd
df = pd.read_csv("/home/jakub/jakluz-DE3.1.5/data/raw/loan-test.csv", )

In [56]:
for column in df.columns:
    print(f"{column}, values: {df[column].unique()[0:20]}")

Loan_ID, values: ['LP001015' 'LP001022' 'LP001031' 'LP001035' 'LP001051' 'LP001054'
 'LP001055' 'LP001056' 'LP001059' 'LP001067' 'LP001078' 'LP001082'
 'LP001083' 'LP001094' 'LP001096' 'LP001099' 'LP001105' 'LP001107'
 'LP001108' 'LP001115']
Gender, values: ['Male' 'Female' nan]
Married, values: ['Yes' 'No']
Dependents, values: ['0' '1' '2' '3+' nan]
Education, values: ['Graduate' 'Not Graduate']
Self_Employed, values: ['No' 'Yes' nan]
ApplicantIncome, values: [ 5720  3076  5000  2340  3276  2165  2226  3881 13633  2400  3091  2185
  4166 12173  4666  5667  4583  3786  9226  1300]
CoapplicantIncome, values: [    0  1500  1800  2546  3422  2400  1516  2916   333  7916  3470  1620
  4380 24000  1250  3750   833  2382   820  1683]
LoanAmount, values: [110. 126. 208. 100.  78. 152.  59. 147. 280. 123.  90. 162.  40. 166.
 124. 131. 200. 300.  48.  28.]
Loan_Amount_Term, values: [360. 240. 180.  nan  60. 480.  84.  12. 300. 350.  36. 120.   6.]
Credit_History, values: [ 1. nan  0.]
Property

I have identified some good candidate columns to be used as dimensions in the star schema. Those are columns with few unique values. Let's store them in a list and create a dictionary with their names as keys, and unique column values as dictionary values.

In [57]:
dim_tables = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Credit_History', 'Property_Area']
dim_df = dict()
for table in dim_tables:
    dim_df[table] = pd.Series(pd.unique(df[table])).dropna()
    print(f"{table} - {dim_df[table].values}")

Gender - ['Male' 'Female']
Married - ['Yes' 'No']
Dependents - ['0' '1' '2' '3+']
Education - ['Graduate' 'Not Graduate']
Self_Employed - ['No' 'Yes']
Credit_History - [1. 0.]
Property_Area - ['Urban' 'Semiurban' 'Rural']


Now, let's replace values in the dimension columns of the fact table with indices from the corresponding values. In SQL, those will be the primary keys in the foreign key relationships. 

In [58]:
fact_table = df
for column in dim_tables:
    fact_table[column].replace(dim_df[column].values, dim_df[column].index, inplace=True)
    print(fact_table[column])

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
362    0.0
363    0.0
364    0.0
365    0.0
366    0.0
Name: Gender, Length: 367, dtype: float64
0      0
1      0
2      0
3      0
4      1
      ..
362    0
363    0
364    1
365    0
366    1
Name: Married, Length: 367, dtype: int64
0      0.0
1      1.0
2      2.0
3      2.0
4      0.0
      ... 
362    3.0
363    0.0
364    0.0
365    0.0
366    0.0
Name: Dependents, Length: 367, dtype: float64
0      0
1      0
2      0
3      0
4      1
      ..
362    1
363    0
364    0
365    0
366    0
Name: Education, Length: 367, dtype: int64
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
362    1.0
363    0.0
364    0.0
365    0.0
366    1.0
Name: Self_Employed, Length: 367, dtype: float64
0      0.0
1      0.0
2      0.0
3      NaN
4      0.0
      ... 
362    0.0
363    0.0
364    NaN
365    0.0
366    0.0
Name: Credit_History, Length: 367, dtype: float64
0      0
1      0
2      0
3      0
4      0
  

In [59]:
fact_table.sample(20)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,0.0,0,0.0,0,0.0,5720,0,110.0,360.0,0.0,0
1,LP001022,0.0,0,1.0,0,0.0,3076,1500,126.0,360.0,0.0,0
2,LP001031,0.0,0,2.0,0,0.0,5000,1800,208.0,360.0,0.0,0
3,LP001035,0.0,0,2.0,0,0.0,2340,2546,100.0,360.0,NaN,0
4,LP001051,0.0,1,0.0,1,0.0,3276,0,78.0,360.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
362,LP002971,0.0,0,3.0,1,1.0,4009,1777,113.0,360.0,0.0,0
363,LP002975,0.0,0,0.0,0,0.0,4158,709,115.0,360.0,0.0,0
364,LP002980,0.0,1,0.0,0,0.0,3250,1993,126.0,360.0,NaN,1
365,LP002986,0.0,0,0.0,0,0.0,5000,2393,158.0,360.0,0.0,2
